In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split

In [2]:
#df_All = pd.read_csv(r"C:\Users\saust\OneDrive - Sasol\1 Project rC4\Jupyter Notebooks\Report 10-20-23 No Fluff\df_All_Avg.csv")

# df_All = pd.read_csv('contData_clean.csv')

#df_All = pd.read_csv(r"C:\Users\saust\OneDrive - Sasol\1 Project rC4\Jupyter Notebooks\Report 10-20-23 No Fluff\df_All_Avg.csv")

# df_All = pd.read_csv('3 Feature Selection/df_Join_all.csv')


df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\3 Feature Selection\contData_all_Avg - 1o2.csv')
df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\3 Feature Selection\contData_all_Avg - 2o2.csv')


#df_All = pd.read_csv(r"C:\Users\saust\OneDrive\Desktop\CodeSpace DLs\102423\Project rC4\3 Final Machine Butanol\df_CDCA6.csv")


#df_All = pd.read_csv(r"C:\Users\saust\OneDrive\Desktop\CodeSpace DLs\102423\Project rC4\3 Final Machine Butanol\df_CDCA6.csv")


In [3]:
# Concatenate (union) the dataframes
df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

print(df_All.head())

    425 %Al    Butanol   Decanol    Ethanol   Hexanol   Octanol   AYC55580  \
0  6.319560  30.366200  1.030260  88.289900  1.365840  1.492120  18.297300   
1  6.319915  30.678500  1.063260  89.685450  1.397135  1.548230  18.040650   
2  6.328340  32.399333  1.647950  88.677200  1.994413  2.454430  16.637633   
3  6.336760  32.845950  1.928805  87.080875  2.288335  2.897523  15.779400   
4  6.337460  33.387025  2.352595  84.489175  2.734682  3.571185  15.199680   

    DI55102   DI55152   DI55580  ...    TI40050   TI52014     TI55013  \
0  0.963347  0.924879  1.017340  ...  81.156900  131.0990  236.672000   
1  0.965043  0.924515  1.014885  ...  82.965650  132.3490  230.501000   
2  0.968107  0.929955  1.015593  ...  82.461167  133.0350  223.885667   
3  0.969573  0.932388  1.014703  ...  83.676300  134.0690  219.719750   
4  0.971022  0.934531  1.014094  ...  85.328040  134.8424  217.253200   

      TI55014     TI55015   TI55016    TI55017  TI55021     TI55023  VI52558B  
0  223.70400

In [4]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [5]:
# df_All = df_All[df_All['Date'] > '2022-06-15 00:00:00']

In [6]:
# # List of columns to exclude to run XGboost feature selection
# exclude_columns = ['Octanol', 'Hexanol', 'Ethanol', 'Decanol',
       
#        'TI52014', 'TI55013', 'TI55014', 'TI55015', 'TI55016', 'TI55017', 'TI55021', 'TI55023',
#        'TC52015', 'FC52018', 'II52554', 'TI40050', 'VI52558B'
#                    ]

# # Create a new DataFrame without the excluded columnsd
# df_All = df_All.drop(columns=exclude_columns)

In [7]:
selected_columns = ['425 %Al', 'Butanol', 'DI55102', 'DI55152', 'DI55580', 'FC42428',
       'FC55003', 'FC55552', 'FC55569', 'FC55576', 'FFC55553', 'FFC55555',
       'LC52572', 'LC55553', 'LC55555', 'LC55557', 'LC55568', 'LC90366',
       'LC90368', 'PI55004', 'PI55020', 'TC55552', 'TC55553', 'TC55555'
                   ]

existing_columns = [col for col in selected_columns if col in df_All.columns]
df_All = df_All[existing_columns]

In [8]:
df_All.columns

Index(['425 %Al', 'Butanol', 'DI55102', 'DI55152', 'DI55580', 'FC42428',
       'FC55003', 'FC55552', 'FC55569', 'FC55576', 'FFC55553', 'FFC55555',
       'LC52572', 'LC55553', 'LC55555', 'LC55557', 'LC55568', 'LC90366',
       'LC90368', 'PI55004', 'PI55020', 'TC55552', 'TC55553', 'TC55555'],
      dtype='object')

In [9]:
# Splitting into train and test
X = df_All.drop('Butanol', axis=1)  # Assuming 'target' is your target column
y = df_All['Butanol']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Create a linear regression model
linreg = LinearRegression()

# Create a SequentialFeatureSelector object
sfs = SFS(linreg, 
          k_features='best', 
          forward=True, 
          floating=True, 
          scoring='r2',
          cv=5)

# Fit the model
sfs = sfs.fit(X_train, y_train)

# Get the selected feature names
selected_features = X_train.columns[list(sfs.k_feature_idx_)]

# Build the final model using selected features
final_model = linreg.fit(X_train[selected_features], y_train)

# Optionally, evaluate the model on the test set
y_pred = final_model.predict(X_test[selected_features])

In [11]:
# Get the selected feature names
selected_features = X_train.columns[list(sfs.k_feature_idx_)]
print("Selected Features:")
print(selected_features)

# Display the coefficients of the final model
print("\nModel Coefficients:")
for i, feature in enumerate(selected_features):
    print(f"{feature}: {final_model.coef_[i]}")

# Display the R-squared value for the training set
r_squared_train = final_model.score(X_train[selected_features], y_train)
print(f"\nR-squared on Training Set: {r_squared_train}")

# Optionally, if you evaluated the model on a test set
r_squared_test = final_model.score(X_test[selected_features], y_test)
print(f"R-squared on Test Set: {r_squared_test}")


Selected Features:
Index(['DI55102', 'DI55152', 'DI55580', 'FC42428', 'FC55003', 'FC55552',
       'FC55569', 'FC55576', 'FFC55553', 'FFC55555', 'LC52572', 'LC55553',
       'LC55555', 'LC55557', 'LC55568', 'LC90366', 'LC90368', 'PI55004',
       'PI55020', 'TC55552', 'TC55553', 'TC55555'],
      dtype='object')

Model Coefficients:
DI55102: 24.98290662827781
DI55152: -3.0520922010203804
DI55580: 13.370015397011752
FC42428: -9.096045437712732e-05
FC55003: -0.0008478237082039891
FC55552: 0.00042723928048791674
FC55569: -0.0021131129132163137
FC55576: 0.006359060746466037
FFC55553: 5.182042417643112
FFC55555: -35.47138220315785
LC52572: 0.1530553021328216
LC55553: -0.158442152949189
LC55555: 0.11881791152727368
LC55557: -0.24050222706510838
LC55568: 0.406071192915321
LC90366: -0.020756542895760236
LC90368: 0.035500959850818116
PI55004: -0.28698322201231535
PI55020: 1.4017173225464035
TC55552: 0.047743984836711724
TC55553: 0.026048755188356216
TC55555: 0.1929901456256612

R-squared on Tra